# Calculating with functions

We are used to performing numerical calculations with numbers, using floating-point approximations of reals that where the approximation differs by machine epsilon from the exact value.

In this lecture we will see that this idea can be extended to do **calculations with functions**, using **approximations of functions** that differ from the exact function by around machine epsilon! 

The basic idea is a simple one: out of all functions, there are a few that we understand well and are able to calculate efficiently with, namely polynomials. So we will use in our manipulations not the original functions, but rather **polynomial approximations** that are machine epsilon-close. With careful choices of method and algorithm, it turns out that this may be done very efficiently, using Chebyshev interpolation and approximation by Chebyshev polynomials.

These ideas were first developed by [Nick Trefethen](https://people.maths.ox.ac.uk/trefethen/) and his group in the [Chebfun](http://www.chebfun.org/) package for Matlab. An implementation in Julia is available in the 
[ApproxFun.jl](https://github.com/JuliaApproximation/ApproxFun.jl) package.

## Naive polynomial interpolation

One way to approximate data is by interpolation. Given $n+1$ data points $(x_i, y_i)_{i=0}^{n}$, there is a unique polynomial $p$ of (maximum) degree $n$ that passes through them. We can find it in a simple (but naive, and not efficient) way as follows:

Let $$p(x) = a_0 + a_1 \, x + a_2 \, x^2 + \cdots + a_n \, x^n$$.

Then we have $n+1$ simultaneous equations 
$$p(x_i) = a_0 + a_1 \, x_i + a_2 \, x_i^2 + \cdots + a_n \, x_i^n = y_i$$
to solve, i.e.

$$
\begin{pmatrix}
1 & x_0 & x_0^2 & \cdots & x_0^n \\
1 & x_1 & x_1^2 & \cdots & x_1^n \\
1 & x_2 & x_2^2 & \cdots & x_2^n \\
\vdots
\\ 
1 & x_n & x_n^2 & \cdots & x_n^n \\
\end{pmatrix}
\cdot 
\begin{pmatrix}
a_0 \\ \vdots \\ a_n
\end{pmatrix}
=
\begin{pmatrix}
y_0 \\ \vdots \\ y_n
\end{pmatrix}
$$

The matrix above is known as a Vandermonde matrix. The code below generalizes to allow for rectangular "Vandermonde" matrices, but defaults to the square case.

In [1]:
vander(x,n=length(x)-1) = x.^(0:n)'

vander (generic function with 2 methods)

In [2]:
vander(1:2,1)

2×2 Array{Int64,2}:
 1  1
 1  2

Given a function $f$, we can choose points $x_i$ and let $y_i = f(x_i)$. It is natural (but, it turns out, incorrect) to try equi-spaced points:

In [3]:
lo, hi = -1, 1  # the interval we work over

(-1, 1)

In [ ]:
n = 10
x = linspace(lo, hi, n+1)
M = vander(x)

In [ ]:
a = M \ exp.(x)   # kind of looks like taylor series, almost, but not exactly 

The entries of `a` are the coefficients of the fitted polynomial. We can now build (again, naively) the interpolating polynomial:

In [4]:
import Base: +, *, -

*(α::Number, f::Function) = x -> α*f(x)
+(f::Function, g::Function) = x -> f(x) + g(x)
-(f::Function, g::Function) = x -> f(x) - g(x)

- (generic function with 195 methods)

In [ ]:
pp = sum(a[i]*(x->x^(i-1)) for i in 1:length(a))
ttk(k) = sum((1/factorial(i))*(x->x^i) for i in k:-1:0)

In [ ]:
using Plots, Interact
plotlyjs()
#PyPlot.svg(true)

In [ ]:
#l = @layout [a b]
#p = plot(f, lo, hi, layout=l,label="function")
@manipulate for k=2:100

plot!(pp, lo, hi, label="interpolant")
plot( x-> log10(abs((f-pp)(x))), lo, hi, label="error of the interpolant")
plot!( x-> log10(abs((f-ttk(10))(x))), lo, hi, label="error in taylor")
end

However, this goes wrong for certain functions:

In [ ]:
h(x) = 1 / (1 + 25x^2)  # the Runge function

In [ ]:
x

In [11]:
function interpolate(f, x)
    lo, hi = -1.0, 1.0
    M = vander(x)
    y = f.(x)

    a = M \ y

    p = sum(a[i]*(x->x^(i-1)) for i in 1:length(a))
        
    return p, y, a
end

interpolate (generic function with 1 method)

In [ ]:
using Interact

In [ ]:
h(x) = 1 / (1 + 25x^2)  # the Runge function

#n = 20
@manipulate for n in 2:20
    x = linspace(-1, 1, n+1)
    p, y = interpolate(h, x)

    plot(x->h(x), lo, hi)
    scatter!(x, y)
    plot!(p, lo, hi, label="Interpolant")
end

## Chebyshev points

This problem can be **eliminated** by choosing interpolation points which **cluster near the boundary** in a certain way. A simple set are the Chebyshev points, defined by projecting down equi-spaced points on the unit circle to the $x$-axis.

In [39]:
using Plots
plotlyjs()

Plots.PlotlyJSBackend()

In [ ]:
@manipulate for n in 2:100

#n = 100
    plot(x->sqrt(1-x^2), -1, 1,ylim=(0,1), aspect_ratio=1,label="")
    
    θs = linspace(0, π, n)
    c,s = cos(θs), sin(θs)
    
    for i=1:length(c)     
        plot!([c[i], c[i]], [0, s[i]], ls=:dot, legend=false, label="T$i")
    end
    
    scatter!(c, s, markersize=1,label="")
    scatter!(c, zeros(s), markersize=1,label="")
end

In [9]:
chebyshev_points(n) = cos.(linspace(0, π, n))

chebyshev_points (generic function with 1 method)

Interpolating in these points is **well-behaved**:

In [ ]:
chebyshev_points(11)

In [7]:
using Interact

INFO: Recompiling stale cache file /Users/edelman/.julia/lib/v0.6/Interact.ji for module Interact.


In [12]:
h(x) = 1 / (1 + 25x^2)  # the Runge function

@manipulate for n in 2:100
    
#n = 5

    x = chebyshev_points(n)
    p, y = interpolate(h, x)

    plot(h, lo, hi, label="Runge function")
    scatter!(x, y)
    plot!(p, lo, hi)
end

Interact.Options{:SelectionSlider,Any}(5: "input-3" = 51 Any , "n", 51, "51", 50, Interact.OptionDict(DataStructures.OrderedDict{Any,Any}("2"=>2,"3"=>3,"4"=>4,"5"=>5,"6"=>6,"7"=>7,"8"=>8,"9"=>9,"10"=>10,"11"=>11…), Dict{Any,Any}(Pair{Any,Any}(68, "68"),Pair{Any,Any}(2, "2"),Pair{Any,Any}(89, "89"),Pair{Any,Any}(11, "11"),Pair{Any,Any}(39, "39"),Pair{Any,Any}(46, "46"),Pair{Any,Any}(85, "85"),Pair{Any,Any}(25, "25"),Pair{Any,Any}(55, "55"),Pair{Any,Any}(42, "42")…)), Any[], Any[], true, "horizontal")

So we see that we should choose the polynomial with which to approximate a function $f$ by **interpolation in Chebyshev points**. The question is how many points we should use?

In [30]:
interpolate(h, chebyshev_points(500))

(#3, [0.0384615, 0.038463, 0.0384674, 0.0384747, 0.038485, 0.0384982, 0.0385144, 0.0385335, 0.0385555, 0.0385805  …  0.0385805, 0.0385555, 0.0385335, 0.0385144, 0.0384982, 0.038485, 0.0384747, 0.0384674, 0.038463, 0.0384615], [0.99796, -0.0606807, -25.2422, 3.37539, 637.624, -129.248, -15200.3, 2414.69, 3.05953e5, 1086.15  …  -2.04367e12, 1.47797e12, 7.94609e10, 3.53537e11, -6.6208e11, 6.21024e11, 1.33862e12, 7.33868e11, -7.55146e10, -1.23212e12])

In [37]:
pp, yy, aa = interpolate(exp, big.(chebyshev_points(200)));

In [38]:
Float64.(aa)

200-element Array{Float64,1}:
  1.0        
  1.0        
  0.5        
  0.166667   
  0.0416667  
  0.00833333 
  0.00138889 
  0.000198413
  2.48016e-5 
  2.75573e-6 
  2.75573e-7 
  2.50521e-8 
  2.08768e-9 
  ⋮          
  9.74733e-15
 -4.84124e-13
 -1.41423e-15
  5.22069e-14
  1.5065e-16 
 -4.4323e-15 
 -1.13443e-17
  2.7779e-16 
  5.44343e-19
 -1.14268e-17
 -1.25871e-20
  2.31416e-19

## Approximation by Chebyshev polynomials

It turns out that the standard monomial basis ($1$, $x$, $x^2$, ...) is **not** a good basis for polynomial interpolation. A good basis is the set of **Chebyshev polynomials**, defined by

$$T_n(x) = \mathrm{Re}(z^n) = \cos(n \theta) = \cos(n \cos^{-1}(x))$$

In [ ]:
T(n) = x -> cos(n * acos(x))  # a simple, but unintuitive way to define these in Julia

We can do polynomial interpolation using Chebyshev polynomials by writing a similar formula to that above:

$$f(x) = \sum_n \alpha_n T_n(x)$$

and again doing naive interpolation:

In [ ]:
chebyshev_points(n) = cos.(linspace(0, π, n))

In [ ]:
function chebyshev_interpolate(f, xx)
    lo, hi = -1.0, 1.0
    
    n = length(xx) - 1

    yy = [f(x) for x in xx]

    M = [T(i)(x) for x in xx, i in 0:n]
    

    aa = M \ yy

    pp = sum(aa[i]*(x->x^(i-1)) for i in 1:length(aa))
        
    return pp, yy, aa
end

In [ ]:
xx = chebyshev_points(25)
pp, yy, aa = chebyshev_interpolate(exp, xx)

In [ ]:
aa

The point is that the coefficients decay quickly (until they reach something of order machine epsilon) for smooth functions:

In [ ]:
scatter(abs.(aa), yscale=:log10)

In [ ]:
n = 11
xx = chebyshev_points(n)
ff = exp.(xx)

In [ ]:
real(fft(ff)) / (n/2)

The idea is thus to adaptively choose the number of points at which the function is sampled, until the tail of the  coefficients in the Chebyshev expansion have decayed to machine precision.

## From data to Chebyshev coefficients

It turns out that there is a key relationship between Chebyshev points and Chebyshev polynomials, which enables us to find the coefficients $\alpha_n$ in the expansion in Chebyshev polynomials in time $\mathcal{O}(n \, \log n)$, instead of the $\mathcal{O}(n^3)$ that the naive matrix formulation requires. This entails the use of the discrete cosine transform, a close relative of the FFT.

The idea is as follows. Suppose that we in fact know the expansion $f = \sum a_n T_n$. Then evaluating this in a Chebyshev point $x_i$ gives

$$f(x_i) = \sum a_k T_k(x_i)$$.

But $x_i = \cos (i \pi / n)$ and $T_k(x) = \cos(k \cos^{-1}(x))$, so

$$f(x_j) = \sum_k a_k \cos(j \, k \pi / n)$$.

That is, the data $f(x_i)$ are given by the **discrete cosine transform** of the coefficients $a_n$. Furthermore, from this we see that we can calculate the $a_n$ by taking the inverse discrete cosine transform of the data. This is very closely related to the FFT and can be similarly calculated in $\mathcal{O}(n \, \log n)$ time:

$$\sum_j f(x_j) \cos(j l\pi/n) = \sum_j \sum_k a_k \cos(j k\pi / n) \cos(j l\pi/n)$$

$$ = \sum_k a_k (n/2) \delta_{kl} = (n/2) a_l$$

In [ ]:
ff

In [ ]:
fft(ff)

In [ ]:
?fft

Discrete cosine transform matrix:

In [ ]:
n = 11
xx = chebyshev_points(n+1)
M = [T(i)(x) for x in xx, i in 0:n]


Inverse discrete cosine transform matrix:

In [ ]:
(n/2)*inv(M)

## Barycentric interpolation

One final piece of the puzzle is how to evaluate the interpolating function at different points $x$. We could construct the whole function as the sum $a_n T_n$, but this is not actually necessary. Instead we use barycentric interpolation [Trefethen 2004]: given data $f_j$ at nodes $x_j$, the value of the interpolating polynomial at $x$ is

$$p(x) = \frac{\sum_j \frac{w_j}{x-x_j} f_j}{\sum_j \frac{w_j}{x - x_j}}$$

The weights are given by 

$$w_j = \frac{1}{\prod_{j \neq k} (x_j - x_k)}$$.

These depend **only** on the interpolation points, not on the data. For Chebyshev points, they are

$$w_j = (-1)^j \delta_j,$$

where $\delta_j = 1/2$ for $j=0$ and $j=n$, and $1$ otherwise.

In [ ]:
function ww(j)
    j -= 1  # 0-based
    sign = iseven(j) ? 1 : -1
    value = (j==0 || j==n) ? 0.5 : 1.0
    
    return sign*value
end

In [ ]:
n = 11
xx = chebyshev_points(n)
g(x) = 1 / (1 + 25x^2)
ff = g.(xx)

In [ ]:
using Plots

In [ ]:
ppp(x) = sum(ww(j) / (x-xx[j]) * ff[j] for j = 1:length(xx)) / sum(ww(j) / (x-xx[j]) for j = 1:length(xx))

In [ ]:
using Interact

In [ ]:
g(x) = 1 / (1 + 25x^2)
n = 15
#@manipulate for n in 2:100
    xx = chebyshev_points(n)
   
    ff = g.(xx)

    plot(x->ppp(x), -1, 1)
    plot!(f, -1, 1)
#end

In [ ]:
@manipulate for i in 1:10
    i^2
end

In [ ]:
using PyCall

In [ ]:
Pkg.add("PyPlot")
Pkg.add("PlotlyJS")

In [ ]:
ENV["PYTHON"]=""
ENV["JUPYTER"] = ""

Pkg.build("PyCall")
Pkg.build("PyPlot")
Pkg.build("IJulia")

In [ ]:
Pkg.build("PyCall")

In [ ]:
Pkg.build("Interact")

In [ ]:
Pkg.build("IJulia")

In [ ]:
using Interact

In [ ]:
@manipulate for i in 1:10
    i^2
end

## ApproxFun

In [ ]:
using ApproxFun

In [ ]:
Fun

In [ ]:
x = Fun()

In [ ]:
# 0*T_0 + 1.0*T_1 = 0 + x = x

In [ ]:
x(0.3)

In [ ]:
x(1.5)

In [ ]:
x^2

In [ ]:
# 0.5 T_0 + 0.5*T_2 = 0.5 + 0.5*(2x^2 - 1) = x^2

In [ ]:
f = sin(x^2)

In [ ]:
f.coefficients

In [ ]:
using Plots

In [ ]:
plot(f, -1, 1)

In [ ]:
f

In [ ]:
f'